# 0. Imports  


In [ ]:
# Zunächst werden die erforderlichen Bibliotheken importiert.
import tensorflow as tf
import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import platform
from keras.callbacks import TensorBoard
from keras.utils import plot_model
from PIL import Image
from datetime import datetime
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
import sys

# Prüfen ob das Notebook lokal oder in Google Colab läuft
IN_COLAB = 'google.colab' in sys.modules

# Download Utility Function if in Colab
if IN_COLAB:
  url = "https://seafile.cloud.uni-hannover.de/f/c185fc5036564611999d/?dl=1"
  filename = "Utility_Functions.zip"
  !curl $url -o $filename -J -L
  !unzip -o -q $filename


from Utility_Functions.MLL_Callback_Functions import PlotHistoryAndExport, log_confusion_matrix, Conv_Resolution #Custom MLL Callback Functions

# Prüfen ob Tensorflow für das Training Zugriff auf die GPU hat.
if not IN_COLAB:
  pysical_devices = tf.config.list_physical_devices("GPU")
  tf.config.experimental.set_memory_growth(pysical_devices[0],True)

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', keras.__version__)

# 1. Laden des Datensatzes

In [ ]:
# Die Variable DeinNachname wird verwendet, um Plots, Daten und die neuronalen Netze unter individuellen Namen zu speichern.
DeinNachname = "Dein_Nachname"           # Hier deinen Nachnamen eintragen

# Prüfen ob das Notebook lokal oder in Google Colab läuft
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  print("IN_COLAB")
  url = "https://seafile.cloud.uni-hannover.de/f/51e1b4397ea444488ae0/?dl=1"
  # Manuelles Festlegen der Archivnamens, da SeaFile diesen nicht mit dem Downloadlink gibt
  DownloadFilename = "match_MLL_dataset_1280.zip"
  DownloadFoldername = "match_MLL_dataset_1280"
  DatasetDirectory = os.path.abspath(DownloadFoldername)
  print(f"Dataset Directory: {DatasetDirectory}")
  DownloadDataset = True

  if os.path.exists(DatasetDirectory):
    overwrite = input(f"Folder {DownloadFoldername} already exists. Overwrite? [y/N]")
    if not overwrite in ["y", "Y"]:
      DownloadDataset = False

  if DownloadDataset:
    # Datesatz herunterladen:
    # os.system(f"curl {url} -o {filename}")
    # Command direkt in Colab ausführen für bessere Ausgabe
    !curl -L $url -o $DownloadFilename
    print("Download complete")
    # Archiv entpacken:
    # os.system(f"unzip -o {filename}")
    # Command direkt in Colab ausführen für bessere Ausgabe
    !unzip -o -q $DownloadFilename
    print("Dataset extracted successfully")

  Directory = os.path.abspath(DownloadFoldername)


else:
  Directory = r'Z:\Datensatz_Archiv\Random_III_AlleKlassen'  # hier den Pfad für den Datensatz angeben

print(f"Directory: {Directory}")
# Float zwischen 0 und 1, Anteil der Daten, die für die Validierung reserviert werden sollen.
Split=0.2                           # hier den Split festlegen

# Die Bilder in unserem Datensatz haben eine Originalgröße bzw. Auflösung von (1536,2048)
# Um die Speichernutzung effizienter zu gestalten, werden wir die Bilder direkt mit reduzierter Auflösung laden,
# anstatt die Bilder mit der Originalauflösung zu laden und diese dann zu reduzieren.
Reduced_Image_Size = [360,480]      # (Höhe, Breite)


# Zunächst werden die Daten angeben, welche bezüglich des überlegten Bildausschnitts relevant sind.
offset_width_GUI = 870           # Startpunkt x-Koordinate
offset_height_GUI = 110          # Startpunkt y-Koordinate
target_size_GUI = (2450,2450)   # (target_height, target_width), die Größe des Bildausschnitts

# Diese werden umgerechnet relativ zu der Göße, mit welcher die Bilder geladen werden sollen.
offset_height = Conv_Resolution(Reduced_Image_Size,offset_height_GUI)         # Startpunkt y-Koordinate
offset_width = Conv_Resolution(Reduced_Image_Size,offset_width_GUI)           # Startpunkt x-Koordinate
target_size = (Conv_Resolution(Reduced_Image_Size,target_size_GUI[0]),Conv_Resolution(Reduced_Image_Size,target_size_GUI[1]))   # (target_height, target_width), die Größe des Bildausschnitts

# Die Input_Image_Shape wird später verwendet, um die Eingabeform für die erste Faltungsschicht zu definieren.
# "size" definiert nur die Höhe und die Breite.
# "shape" hat zusätzlich die Anzahl an Farbkanälen (3 für RGB, 1 für Greyscale).
# Der abgeschnittene Teil des Bildes wird als Eingabe verwendet.
# Input_Image_Shape = (target_size[0], target_size[1], 3)
Input_Image_Shape = (224, 224, 3)

In [ ]:
# Jetzt laden wir den Datensatz

# 2. Formating, Cropping and Resizeing

In [ ]:
# Hier soll eine Funktion definiert werden, die die Pixelwerte eines Bildes in Floats zwischen 0 und 1 umwandelt
# und das Bild auf die gewünschte Größe, wie oben definiert, zuschneidet.
# Hinweis 1: Die Funktion format_example im Tutorial macht fast das, was wir wollen.
# Der Unterschied besteht darin, dass wir nicht die Größe des Bildes reduzieren, sondern einen Teil des Bildes zuschneiden wollen.
# Hinweis 2: google tf.image.crop_to_bounding_box()
# vergiss nicht, die Funktion map() zu verwenden, um die Formatierung auf alle Bilder des Datensatzes anzuwenden.

# 3. Image-Augmentation

In [ ]:
# Hier sollten die Funktionen zur Data-Augmentation definiert werden.

# 4. Show Pictures

# 5. Shuffle and Batching

In [ ]:
BATCH_SIZE = 30  # Hier wird die Batchgröße festgelegt    #Default 30

# 6. Create Model

In [ ]:
# Erstellt ein neues Sequenzielles Modell mit dem Variablenname "model"
model = keras.Sequential()

model.summary()

# 7. Compile Model

# 8. Callbacks - Trainingshistorie und Confusion-Matrix


In [ ]:
# Hier definieren, falls abweicht!!
######
Modellname = model  # Hier Modellobjektnamen angeben
Validationset = dataset_test_shuffled  # Ihr den Validationsdatensatz angeben
#######

NAME_der_LogDatei = "Neuronales_Netz_{}".format(
    datetime.now().strftime("%Y%m%d-%H_%M_%S")
)  # Hier wird der Name der Logdatein definiert

if IN_COLAB:
    TensorBoard_Log_Image_PATH = "logs/{}/{}/Image_{}".format(
        DeinNachname, NAME_der_LogDatei, NAME_der_LogDatei
    )
    TensorBoard_Log_Train_History_PATH = "logs/{}/{}".format(
        DeinNachname, NAME_der_LogDatei
    )
else:
    TensorBoard_Log_Image_PATH = "Z:/Programmierung_NN/logs/{}/{}/Image_{}".format(
        DeinNachname, NAME_der_LogDatei, NAME_der_LogDatei
    )
    TensorBoard_Log_Train_History_PATH = "Z:/Programmierung_NN/logs/{}/{}".format(
        DeinNachname, NAME_der_LogDatei
    )

TensorBoard_callback_Train_History = TensorBoard(
    log_dir=TensorBoard_Log_Train_History_PATH
)

file_writer_cm = tf.summary.create_file_writer(TensorBoard_Log_Image_PATH)


class MLLCallbacks(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        PlotHistoryAndExport(TensorBoard_Log_Train_History_PATH)
        model = tf.keras.models.load_model(
            TensorBoard_Log_Train_History_PATH + "/Best_model.h5"
        )
        log_confusion_matrix(
            None,
            Validationset,
            model,
            class_names,
            file_writer_cm,
            TensorBoard_Log_Train_History_PATH,
            logs,
            "yes",
        )
        # log_confusion_matrix(None,Validationset, Modellname, class_names, file_writer_cm, TensorBoard_Log_Train_History_PATH, logs,'yes')

    def on_epoch_end(self, epoch, logs=None):
        if (epoch % 1) == 0:
            log_confusion_matrix(
                epoch,
                Validationset,
                Modellname,
                class_names,
                file_writer_cm,
                TensorBoard_Log_Train_History_PATH,
                logs,
                "no",
            )


# stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=0,verbose=0,mode='auto',baseline=None,restore_best_weights=False)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    TensorBoard_Log_Train_History_PATH + "/Best_model.h5",
    monitor="val_loss",
    save_best_only=True,
)

# Funktion zum Plotten von Netzstrukturen.
# Das Bild wird auch gespeichert.
# Für verschiedene Modelle den Wert des Parameters to_file anpassen. Bsp.: f'{DeinNachname}_1.png'
plot_model(
    model,
    to_file=f"{TensorBoard_Log_Train_History_PATH}/{DeinNachname}.png",
    show_shapes=True,
    show_layer_names=True,
)

# 9. Training

In [ ]:
train_epoches = 20  # Festlegen der Trainingsepochen

# Die Funktion fit() mit den benötigten Parametern anpassen.
# Den Parameter callbacks nicht löschen wenn local trainiert wird.
# Bei Ausführung in Colab kann der Parameter callbacks gelöscht/auskommentiert werden.
# Durch den langsmaen file i/o speed auf colab verlangsamt dieser das Training erheblich.
training_history = model.fit(
    x=dataset_train_augmented_shuffled.repeat(),  # Festlegen des Trainingsdatensets
    validation_data=dataset_test_shuffled.repeat(),  # Festlegen des Validationdatensets
    epochs=train_epoches,
    steps_per_epoch=steps_per_epoch,  # Muss nur definiert werden, wenn dataset.repeat() in fit() verwendet wird, damit der Trainingsalgorithmus eine Stop Condition hat.
    validation_steps=validation_steps,  # Muss nur definiert werden, wenn dataset.repeat() in fit() verwendet wird
    verbose=1,
    callbacks=[TensorBoard_callback_Train_History, MLLCallbacks()],
)
# Get best Model after Training
model = tf.keras.models.load_model(
    TensorBoard_Log_Train_History_PATH + "/Best_model.h5"
)

# 10. Start Tensorboard

In [ ]:
#cd Z:\Programmierung_NN
#tensorboard --logdir=logs/NACHNAME/

if IN_COLAB:
    %load_ext tensorboard
    %tensorboard --logdir logs/$deinNachname/
else:
    Command='start cmd /k "Z: & cd Z:\Programmierung_NN & tensorboard --logdir=logs/'+DeinNachname+'/"'
    os.system(Command)

#Öffne http://localhost:6006/ in deinem Browser wenn nicht in colab

# 11. Export als tf.lite Modell

In [ ]:
# falls der Kernel mal abstürzen sollte:
# model = tf.keras.models.load_model(Pfad der log datei)

if IN_COLAB:
    Lite_model_file_name = f"/content/MLL-Netz_{DeinNachname}"
    model_file_path = f"/content/{DeinNachname}"
else:
    Lite_model_file_name = f"Z:/Netze/MLL-Netz_{DeinNachname}"
    model_file_path = f"Z:/Netze/{DeinNachname}"

# save models
converter = tf.lite.TFLiteConverter.from_keras_model(
    model
)  # hier den Modellobjektnamen eintragen
tflite_model = converter.convert()
open(Lite_model_file_name + ".tflite", "wb").write(tflite_model)
tf.keras.models.save_model(model, f"{DeinNachname}")

# 12. Download der trainierten Modelle

In [ ]:
# Download models to local machine, only necessary if in colab
if IN_COLAB:
    from google.colab import files
    # Compress model for download
    !tar -czvf model.tar.gz $DeinNachname
    files.download('model.tar.gz')
    files.download(Lite_model_file_name + '.tflite')